In [1]:
# aggtrades_fixed_range.py
import requests, time
from datetime import datetime, timezone
import polars as pl

BASE = "https://api.binance.com"

# ==== CONFIG: set fixed UTC range here ====
SYMBOL = "BNBUSDT"
START  = "2024-01-01T00:00:00Z"  # UTC
END    = "2024-01-01T00:05:00Z"  # UTC
# =========================================

def to_ms(dt: datetime) -> int:
    return int(dt.timestamp() * 1000)

def parse_iso_utc(s: str) -> datetime:
    # Accept "YYYY-MM-DDTHH:MM:SS[.fff][Z]"
    if s.endswith("Z"):
        s = s.replace("Z", "+00:00")
    dt = datetime.fromisoformat(s)
    if dt.tzinfo is None:
        dt = dt.replace(tzinfo=timezone.utc)
    return dt.astimezone(timezone.utc)

def iso(ms: int) -> str:
    return datetime.fromtimestamp(ms/1000, tz=timezone.utc).isoformat()

def fetch_aggtrades_range(symbol: str, start_ms: int, end_ms: int, limit=1000, max_pages=10000):
    """
    Phân trang theo thời gian: mỗi vòng gọi với startTime=cursor, endTime=end_ms.
    Khi đầy trang (limit), dịch cursor = last.T + 1ms và lặp đến khi qua end_ms.
    """
    url = f"{BASE}/api/v3/aggTrades"
    out = []
    cursor = start_ms
    for _ in range(max_pages):
        if cursor > end_ms:
            break
        params = {"symbol": symbol, "startTime": cursor, "endTime": end_ms, "limit": limit}
        r = requests.get(url, params=params, timeout=10)
        r.raise_for_status()
        batch = r.json()
        if not batch:
            break
        out.extend(batch)
        last_T = batch[-1]["T"]
        # Nếu server trả về cùng một mốc T ở cuối (edge case), nảy 1ms để tránh lặp vô hạn
        cursor = last_T + 1
        # Một chút nghỉ để lịch sự với API (tuỳ chọn)
        # time.sleep(0.01)
        if len(batch) < limit and last_T >= end_ms:
            break
    return out

def aggtrades_to_1m_orderflow(rows):
    """
    Tạo bảng 1m với:
      buy_count/sell_count, buy_base/sell_base, buy_quote/sell_quote
    Quy ước:
      m == False -> buyer là taker  -> TAKER BUY
      m == True  -> buyer là maker  -> TAKER SELL
    """
    if not rows:
        return pl.DataFrame({
            "open_time": pl.Series([], dtype=pl.Datetime),
            "buy_count": pl.Series([], dtype=pl.Int64),
            "sell_count": pl.Series([], dtype=pl.Int64),
            "buy_base":  pl.Series([], dtype=pl.Float64),
            "sell_base": pl.Series([], dtype=pl.Float64),
            "buy_quote": pl.Series([], dtype=pl.Float64),
            "sell_quote":pl.Series([], dtype=pl.Float64),
        })

    df = pl.DataFrame({
        "T": [r["T"] for r in rows],    # time (ms)
        "m": [r["m"] for r in rows],    # buyer is maker
        "p": [r["p"] for r in rows],    # price (string)
        "q": [r["q"] for r in rows],    # qty   (string)
    }).with_columns([
        pl.from_epoch(pl.col("T")/1000, time_unit="s").alias("ts"),
        pl.col("p").cast(pl.Float64).alias("price"),
        pl.col("q").cast(pl.Float64).alias("qty"),
    ]).with_columns([
        pl.col("ts").dt.truncate("1m").alias("open_time"),
        (pl.col("price") * pl.col("qty")).alias("quote"),
    ])

    out = df.group_by("open_time").agg([
        (~pl.col("m")).sum().alias("buy_count"),
        ( pl.col("m")).sum().alias("sell_count"),
        pl.when(~pl.col("m")).then(pl.col("qty")).otherwise(0.0).sum().alias("buy_base"),
        pl.when( pl.col("m")).then(pl.col("qty")).otherwise(0.0).sum().alias("sell_base"),
        pl.when(~pl.col("m")).then(pl.col("quote")).otherwise(0.0).sum().alias("buy_quote"),
        pl.when( pl.col("m")).then(pl.col("quote")).otherwise(0.0).sum().alias("sell_quote"),
    ]).sort("open_time")

    return out




In [2]:
start_ms = to_ms(parse_iso_utc(START))
end_ms   = to_ms(parse_iso_utc(END))
assert end_ms > start_ms, "END must be after START"

rows = fetch_aggtrades_range(SYMBOL, start_ms, end_ms, limit=1000)
print(f"[{SYMBOL}] aggTrades rows: {len(rows)} | {iso(start_ms)} → {iso(end_ms)}")

# In vài dòng raw để xem structure
for r in rows[:5]:
    print(f"a={r['a']} T={iso(r['T'])} p={r['p']} q={r['q']} m(buyerIsMaker)={r['m']}")

# Gộp về 1m orderflow
of_1m = aggtrades_to_1m_orderflow(rows)
print("\n== 1m orderflow ==")
print(of_1m)
if of_1m.height:
    print("\nHead:")
    print(of_1m.head(3))

[BNBUSDT] aggTrades rows: 721 | 2024-01-01T00:00:00+00:00 → 2024-01-01T00:05:00+00:00
a=515330094 T=2024-01-01T00:00:00.107000+00:00 p=311.90000000 q=0.02200000 m(buyerIsMaker)=False
a=515330095 T=2024-01-01T00:00:00.210000+00:00 p=311.90000000 q=1.34900000 m(buyerIsMaker)=False
a=515330096 T=2024-01-01T00:00:00.237000+00:00 p=311.90000000 q=0.26700000 m(buyerIsMaker)=False
a=515330097 T=2024-01-01T00:00:00.357000+00:00 p=311.90000000 q=0.04900000 m(buyerIsMaker)=False
a=515330098 T=2024-01-01T00:00:00.493000+00:00 p=311.90000000 q=0.16400000 m(buyerIsMaker)=False

== 1m orderflow ==
shape: (5, 7)
┌─────────────────────┬───────────┬────────────┬──────────┬───────────┬────────────┬────────────┐
│ open_time           ┆ buy_count ┆ sell_count ┆ buy_base ┆ sell_base ┆ buy_quote  ┆ sell_quote │
│ ---                 ┆ ---       ┆ ---        ┆ ---      ┆ ---       ┆ ---        ┆ ---        │
│ datetime[μs]        ┆ u32       ┆ u32        ┆ f64      ┆ f64       ┆ f64        ┆ f64        │
╞══